In [ ]:
# !pip install -q torch transformers sentencepiece protobuf gradio

import gradio as gr

import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

In [ ]:
INFERENCE_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = 'openlm-research/open_llama_3b_v2'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH, torch_dtype=torch.float16, device_map=INFERENCE_DEVICE,
)

In [ ]:
def generate_text_block(project_description: str, output_type: str) -> dict:
    prompts_by_output_type = {
        "problem": "answer, what is the key problem this service solves?",
        "target_audience": "answer, what is the target audience of this service?",
        "value_proposition": "answer, what is the value proposiion of this service?",
        "description": "summarise the description of this service",
        "solution": "describe, how the service works",
        "competitors": "point its key competitors",
        "founders": "point, who are its founders and investors?",
        "investments_direction": "imagine, what features can be added to service?",
        "roadmap": "imagine the roadmap of service development"
    }
    prompt = f"Text: {project_description}\nGiven this information, {prompts_by_output_type[output_type]}\n"
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(INFERENCE_DEVICE)

    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=32
    )
    return tokenizer.decode(generation_output[0])[len(prompt)+2:].rstrip('</s>')


def generate_pitch_deck(project_description: str, custom_feature: bool):
    output_types = [
        'problem', 
        'target_audience', 
        'value_proposition', 
        'description', 
        'solution', 
        'competitors', 
        'founders', 
        'investments_direction', 
        'roadmap'
    ]
    project_description = project_description.rstrip()
    pitch_deck_text_blocks = {
        output_type: generate_text_block(project_description, output_type) 
        for output_type in output_types
    }
    
    if custom_feature:
        pass
    
    return "Done"
    
    
demo = gr.Interface(
    fn = generate_pitch_deck,
    inputs = [
        gr.Textbox(lines=2, placeholder="Project Description Here..."),
        gr.Checkbox(label="Some custom feature")
    ],
    outputs = "file",
)
demo.launch()
